# ETL

En esta jupyter notebook se desarrolla la extracción, transformación y carga de los conjuntos de datos.

# Importaciones

In [1]:
import pandas as pd
import json
import gzip
import numpy as np
import ast
from tqdm import tqdm


%load_ext autoreload
%autoreload 2
import utils

import warnings
warnings.filterwarnings("ignore")

# Extracción de los datos

Funcion para extraer los datos ".json" y convertirlos en ".csv" para la exploración.

In [6]:

def procesar_archivo_json(ruta, tipo='json', nombre_archivo_salida='datos.csv'):
    """
    Esta función lee datos desde cada archivo de mi carpeta y los convierte en un DataFrame de pandas.

    Args:
        ruta (str): La ruta al archivo que se va a leer.
        tipo (str, optional): El formato de los datos en el archivo. Puede ser 'json' o 'literal'. 
                              Si es 'json', los datos se cargarán utilizando json.loads. 
                              Si es 'literal', los datos se cargarán utilizando ast.literal_eval. 
                              Por defecto es 'json'.
        nombre_archivo_salida (str, optional): El nombre del archivo CSV de salida.
                                                Por defecto es 'datos.csv'.

    Returns:
        None
    """
    filas = []  # Será la lista para almacenar cada fila de datos

    with gzip.open(ruta, 'rt', encoding='utf-8') as f:
        for line in tqdm(f.readlines(), desc=f"Leyendo {ruta}"):
            if tipo == 'json':
                data = json.loads(line)
            elif tipo == 'literal':
                data = ast.literal_eval(line)
            filas.append(data)

    # Convierte la lista de filas en un DataFrame de pandas
    df = pd.DataFrame(filas)
    
    # Guarda el DataFrame como un archivo CSV
    df.to_csv(nombre_archivo_salida, index=False)

In [42]:
procesar_archivo_json(r'C:\Users\123la\Documents\GitHub\ssss\PI01_Steam_MLops\data\steam_games.json.gz', nombre_archivo_salida='games.csv' )
procesar_archivo_json(r'C:\Users\123la\Documents\GitHub\ssss\PI01_Steam_MLops\data\users_items.json.gz', tipo= 'literal',nombre_archivo_salida='items.csv' )
procesar_archivo_json(r'C:\Users\123la\Documents\GitHub\ssss\PI01_Steam_MLops\data\user_reviews.json.gz', tipo= 'literal', nombre_archivo_salida='reviews.csv')

Leyendo C:\Users\123la\Documents\GitHub\ssss\PI01_Steam_MLops\data\steam_games.json.gz: 100%|██████████| 120445/120445 [00:00<00:00, 143771.85it/s]


# Transformación de los datos


## 'games.csv'.


In [43]:
df = pd.read_csv(r'C:\Users\123la\Documents\GitHub\ssss\PI01_Steam_MLops\data\games.csv', encoding = "utf-8")
df.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
cantidad_vacios_por_columna = df.isnull().sum()
cantidad_total_vacios = df.isnull().sum().sum()

print("Cantidad de valores vacíos por columna:")
print(cantidad_vacios_por_columna)
print("\nCantidad total de valores vacíos en el DataFrame:", cantidad_total_vacios)


Cantidad de valores vacíos por columna:
publisher       96381
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64

Cantidad total de valores vacíos en el DataFrame: 1169016


In [45]:
df = df.dropna()
cantidad_vacios_por_columna = df.isnull().sum()
cantidad_total_vacios = df.isnull().sum().sum()

print("Cantidad de valores vacíos por columna:")
print(cantidad_vacios_por_columna)
print("\nCantidad total de valores vacíos en el DataFrame:", cantidad_total_vacios)


Cantidad de valores vacíos por columna:
publisher       0
genres          0
app_name        0
title           0
url             0
release_date    0
tags            0
reviews_url     0
specs           0
price           0
early_access    0
id              0
developer       0
dtype: int64

Cantidad total de valores vacíos en el DataFrame: 0


##### Que columnas conservar


Teniendo en cuenta que se nos pide : 

* def *developer*( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora.
Mantener : "tags", "developer"

* def *userdata*( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.
Mantener : "price"

* def *UserForGenre*( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.
Mantener : "genre", "release_date"

* def *best_developer_year*( año : int ): Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)
Mantener : "developer"

* def *developer_reviews_analysis*( desarrolladora : str ): Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.
Mantener : "developer".

Las columnas a mantener: "developer", "price", "genre", "release_date", "tags", "id", "title". A eliminar: "app_name", "publisher", "url", "early_access", "specs"

In [46]:
del df['app_name']
del df['publisher']
del df['url']
del df['early_access']
del df['specs']

In [47]:
df.columns

Index(['genres', 'title', 'release_date', 'tags', 'reviews_url', 'price', 'id',
       'developer'],
      dtype='object')

In [48]:
df.head()

,genres,title,release_date,tags,reviews_url,price,id,developer
88310,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,4.99,761140.0,Kotoshiro
88311,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",http://steamcommunity.com/app/643980/reviews/?...,Free To Play,643980.0,Secret Level SRL
88312,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,2017-07-24,"['Free to Play', 'Simulation', 'Sports', 'Casu...",http://steamcommunity.com/app/670290/reviews/?...,Free to Play,670290.0,Poolians.com
88313,"['Action', 'Adventure', 'Casual']",弹炸人2222,2017-12-07,"['Action', 'Adventure', 'Casual']",http://steamcommunity.com/app/767400/reviews/?...,0.99,767400.0,彼岸领域
88315,"['Action', 'Adventure', 'Simulation']",Battle Royale Trainer,2018-01-04,"['Action', 'Adventure', 'Simulation', 'FPS', '...",http://steamcommunity.com/app/772540/reviews/?...,3.99,772540.0,Trickjump Games Ltd


In [49]:
#Convertimos la columa release_date a formato fecha
df['release_date']=pd.to_datetime(df['release_date'], errors='coerce', exact=False)

#Solo nos interesa el año

# Extraer el año y crear una nueva columna 'release_year'
df['year'] = df['release_date'].dt.year.astype('Int64')

# Eliminar la columna 'release_date'
df = df.drop(columns=['release_date'])

In [50]:
# Si aún hay valores nulo, se llenan con la mediana. 
print('cantidad de nulos en "year" :', df['year'].isna().sum())
df['year'] = df['year'].fillna(df['year'].median())
print('cantidad de nulos en "year" :', df['year'].isna().sum())


cantidad de nulos en "year" : 34
cantidad de nulos en "1year" : 0


In [52]:
# Convertir 'price' a tipo numérico, asignar NaN a 'Free To Play'
df['price'] = pd.to_numeric(df['price'], errors='coerce')

In [53]:
df.to_csv('steam_games_cleaned.csv', index=False)
df.to_json('steam_games_cleaned.json', orient='records', lines=True)
df.to_parquet('steam_games_cleaned.parquet', index=False)

## 'items.csv

In [7]:
df = pd.read_csv(r'C:\Users\123la\Documents\GitHub\ssss\PI01_Steam_MLops\data\items.csv')
df.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [8]:
print("\nEstructura de la columna 'items':")
print(df['items'].iloc[0])


Estructura de la columna 'items':
[{'item_id': '10', 'item_name': 'Counter-Strike', 'playtime_forever': 6, 'playtime_2weeks': 0}, {'item_id': '20', 'item_name': 'Team Fortress Classic', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '30', 'item_name': 'Day of Defeat', 'playtime_forever': 7, 'playtime_2weeks': 0}, {'item_id': '40', 'item_name': 'Deathmatch Classic', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '50', 'item_name': 'Half-Life: Opposing Force', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '60', 'item_name': 'Ricochet', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '70', 'item_name': 'Half-Life', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '130', 'item_name': 'Half-Life: Blue Shift', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '300', 'item_name': 'Day of Defeat: Source', 'playtime_forever': 4733, 'playtime_2weeks': 0}, {'item_id': '240', 'item_name': 'Counter-Strike: Source', 'playtime_forever': 18

In [9]:
print(df['items'].dtype)

object


### Datos de la Columna "items"

- **item_id**: El identificador único del ítem.
- **item_name**: El nombre del ítem.
- **playtime_forever**: El tiempo total de juego (en minutos) del usuario para este ítem.
- **playtime_2weeks**: El tiempo de juego (en minutos) del usuario en las últimas dos semanas para este ítem.

"items" parece ser una lista de diccionarios en formato json, se procede a desarmar y agregar cada diccionario como columna al df principal

In [10]:
# Suponiendo que 'items' es la columna que contiene las listas de diccionarios como cadenas
for index, row in df.iterrows():
    # Convertir la cadena que representa el diccionario en un diccionario real
    item_list = ast.literal_eval(row['items'])
    # Iterar sobre cada diccionario en la lista
    for item_dict in item_list:
        # Iterar sobre cada par clave-valor en el diccionario
        for key, value in item_dict.items():
            # Agregar las claves del diccionario como columnas
            df.at[index, key] = value

# Eliminar la columna 'items' original si ya no se necesita
df.drop(columns=['items'], inplace=True)

In [11]:
df.head()

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,273350,Evolve Stage 2,58.0,0.0
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,397060,Faeria,0.0,0.0
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,466170,Idling to Rule the Gods,28545.0,1554.0
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,379720,DOOM,1.0,1.0
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,324810,TOXIKK,0.0,0.0


In [12]:
del df['playtime_2weeks']

In [14]:
df.to_csv('data/user_items_limpio.csv', index=False)
df.to_json('data/user_items_limpio.json', orient='records', lines=True)
df.to_parquet('data/user_items_limpio.parquet', index=False)

## 'reviews.csv'

In [15]:
# Importamos pandas para el análisis de datos tabulares
import pandas as pd

# NumPy proporciona soporte para arreglos y matrices multidimensionales
import numpy as np

# El módulo os permite interactuar con el sistema operativo
import os

# Gdown facilita la descarga de archivos desde Google Drive utilizando su ID
import gdown

# JSON es un formato común para el intercambio de datos, y Python tiene soporte incorporado para trabajar con JSON
import json

# Langdetect es una biblioteca para detectar automáticamente el idioma en el que está escrito un texto
from langdetect import detect

# NLTK (Natural Language Toolkit) es una plataforma para construir programas Python para trabajar con datos de lenguaje humano
import nltk

# SentimentIntensityAnalyzer es una herramienta en NLTK para análisis de sentimientos
from nltk.sentiment import SentimentIntensityAnalyzer



In [2]:
#⚠️Nota: Este comando descarga el modelo de análisis de sentimiento de NLTK y solo debe ejecutarse la primera vez que se utiliza.
#nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\123la\AppData\Roaming\nltk_data...


True

In [26]:
df = pd.read_csv(r'C:\Users\123la\Documents\GitHub\ssss\PI01_Steam_MLops\data\reviews.csv', encoding = "utf-8")
print("Primeras filas del DataFrame:")
df.head()

Primeras filas del DataFrame:


,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [27]:

# Convertir las cadenas JSON en objetos Python
df['reviews'] = df['reviews'].apply(json.loads)

# # Expandir la estructura de los datos JSON y agregar al DataFrame
# df_reviews = pd.json_normalize(df['reviews'])

# # Agregar las columnas 'user_id' y 'user_url' al DataFrame resultante
# df_reviews['user_id'] = df['user_id']
# df_reviews['user_url'] = df['user_url']

# # Definir el orden deseado de las columnas
# column_order = ['user_id', 'user_url', 'funny', 'posted', 'last_edited', 'item_id', 'helpful', 'recommend', 'review']

# # Reordenar las columnas del DataFrame
# df_reviews = df_reviews[column_order]

# # Mostrar las primeras filas del DataFrame resultante
# print(df_reviews.head())

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 3 (char 2)

In [19]:
import pandas as pd
import json

# Replace single quotes with double quotes in the 'reviews' column
df['reviews'] = df['reviews'].str.replace("'", '"')

# Convert 'reviews' column strings to Python objects
df['reviews'] = df['reviews'].apply(json.loads)

# Define the desired order of columns
column_order = ['user_id', 'user_url', 'funny', 'posted', 'last_edited', 'item_id', 'helpful', 'recommend', 'review']

# Iterate over each row, normalize the 'reviews' column, and concatenate the results
result_frames = []
for index, row in df.iterrows():
    reviews_data = row['reviews']
    df_UserReviews = pd.json_normalize(reviews_data, meta=['user_id', 'user_url'])
    result_frames.append(df_UserReviews)

# Concatenate the results into a final DataFrame
final_df = pd.concat(result_frames, ignore_index=True)

# Reorder the columns
df = final_df[column_order]

# Display the final DataFrame
print(df.head())

JSONDecodeError: Expecting value: line 1 column 134 (char 133)